## Prepare Callbacks

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Upwork/Projects/Diabetic-Retinopathy-Prediction/notebooks/diabetic_retinopathy'

In [3]:
os.chdir('../../')

In [4]:
%pwd

'/home/utpal108/dev/Upwork/Projects/Diabetic-Retinopathy-Prediction'

In [5]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from diabeticRetinopathy.constants import *
from diabeticRetinopathy.utils import create_directories, read_yaml

In [7]:
# Configuration Manager

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) ->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), Path(model_ckpt_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = config.checkpoint_model_filepath
        )

        return prepare_callbacks_config

In [8]:
import time
import tensorflow as tf

2024-04-16 09:00:20.083644: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 09:00:20.087031: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 09:00:20.128936: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 09:00:20.825442: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
# Component
class PrepareCallback:
    def __init__(self, config=PrepareCallbacksConfig):
        self.config = config

    # Tensorboard Callback
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    # ModelCheckpoint Callback
    @property
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    # EarlyStopping Callback
    @property
    def _create_estp_callback(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            mode="auto"
        )

    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callback,
            self._create_estp_callback
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callback()

except Exception as e:
    raise e

2024-04-16 09:00:21,521 : diabeticRetinopathy.logger - INFO - YAML file: config/config.yaml loaded successfully
2024-04-16 09:00:21,523 : diabeticRetinopathy.logger - INFO - YAML file: params.yaml loaded successfully
2024-04-16 09:00:21,524 : diabeticRetinopathy.logger - INFO - created directory at: artifacts
2024-04-16 09:00:21,524 : diabeticRetinopathy.logger - INFO - created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
2024-04-16 09:00:21,525 : diabeticRetinopathy.logger - INFO - created directory at: artifacts/prepare_callbacks/checkpoint_dir
